In [1]:
%load_ext autoreload
%autoreload 2
import os

import torch
from torch.utils.data import DataLoader
from datasets import TransformedRoboEireanData, RoboEireanDataWithEncoder
import utils
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from models import MultiClassJetNet
import torchvision.transforms as T
torch.manual_seed(2)

In [2]:
torch.cuda.is_available()

True

In [3]:
# # Default scalings for the default boxes determined by k-means clustering
# default_box_scalings = torch.tensor(
#     [
#         [0.06549374, 0.12928654],
#         [0.11965626, 0.26605093],
#         [0.20708716, 0.38876095],
#         [0.31018215, 0.47485098],
#         [0.415882, 0.8048184],
#         [0.7293086, 0.8216225],
#     ]
# )

default_box_scalings = torch.tensor(
    [
       [0.07817151, 0.08758757],
       [0.12295947, 0.196875  ],
       [0.2434896 , 0.2568398 ],
       [0.35460067, 0.5065973 ],
       [0.39320913, 0.9054488 ],
       [0.97875   , 0.96083343]
    ]
)

classes = ["robot"]
encoder = utils.Encoder(default_box_scalings, ["robot"])
transformed_train_data = TransformedRoboEireanData(
    os.path.join("data", "transformed", "train"), encoder
)
transformed_val_data = TransformedRoboEireanData(
    os.path.join(
        "data",
        "transformed",
        "val",
    ),
    encoder,
)


image_transforms = T.Compose(
            [
                T.Grayscale(),
                T.PILToTensor(),
                T.ConvertImageDtype(torch.float32),
                T.Resize((60, 80)),
            ]
        )
bounding_box_transforms = T.Compose([])

raw_train_data = RoboEireanDataWithEncoder(os.path.join("data", "coco_ball_nao", "train"),
                                           encoder,
                                           ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)

raw_val_data = RoboEireanDataWithEncoder(os.path.join("data", "coco_ball_nao", "val"),
                                           encoder,
                                           ["robot"], image_transforms=image_transforms, bounding_box_transforms=bounding_box_transforms)


In [5]:
learning_rate = 1e-3
train_loader = DataLoader(
    raw_train_data, batch_size=32, shuffle=True, num_workers=0
)
val_loader = DataLoader(
    raw_val_data, batch_size=32, shuffle=False, num_workers=0
)

pl_model = MultiClassJetNet(len(classes), default_box_scalings.size(0), learning_rate)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=200, callbacks=[RichProgressBar()]
)
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy            │ MulticlassAccuracy │      0 │
│ 1 │ input_layer         │ NormConv2dReLU     │    146 │
│ 2 │ depth_wise_backbone │ Sequential         │  4.9 K │
│ 3 │ classifier          │ Sequential         │ 20.9 K │
│ 4 │ output_layer        │ Conv2d             │    900 │
└───┴─────────────────────┴────────────────────┴────────┘

Trainable params: 26.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 26.8 K                                                                                               
Total estimated model params size (MB): 0

Output()

/home/dntai/Documents/GitHub/robot_detection/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.p
y:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), 
RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and 
Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), 
antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on
Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the 
models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(